In [203]:
#anom_test = pd.read_csv('anomalous_traffic_test.csv',sep=',',header=None,\
#                       names=["index","method","url","protocol","userAgent","pragma","cacheControl",\
#                             "accept","acceptEncoding","acceptCharset","acceptLanguage","host","connection",\
#                            "contentLength","contentType","cookie","payload","label","junk1","junk2"])
anom_test = pd.read_csv('anomalous_traffic_test.csv',encoding = 'UTF-8',sep=',',error_bad_lines=False)

b'Skipping line 12771: expected 18 fields, saw 20\nSkipping line 12784: expected 18 fields, saw 20\nSkipping line 12797: expected 18 fields, saw 20\n'
b'Skipping line 73077: expected 18 fields, saw 20\nSkipping line 73090: expected 18 fields, saw 20\nSkipping line 73103: expected 18 fields, saw 20\n'
/Users/jude/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [174]:
normal_test = pd.read_csv('normal_traffic_test.csv',encoding='UTF-8',sep=',',error_bad_lines=False)

b'Skipping line 16706: expected 18 fields, saw 19\n'
b'Skipping line 35368: expected 18 fields, saw 20\nSkipping line 50062: expected 18 fields, saw 20\n'
b'Skipping line 73322: expected 18 fields, saw 19\nSkipping line 85964: expected 18 fields, saw 20\nSkipping line 95918: expected 18 fields, saw 20\n'
/Users/jude/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [100]:
normal_train = pd.read_csv('normal_traffic_training.csv',encoding='UTF-8',sep=',',error_bad_lines=False)

b'Skipping line 6932: expected 18 fields, saw 19\nSkipping line 23588: expected 18 fields, saw 19\n'
b'Skipping line 66700: expected 18 fields, saw 19\nSkipping line 77984: expected 18 fields, saw 19\n'
/Users/jude/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [228]:
anom_test.payload = anom_test.payload.map(str)
anom_test_payload = pd.DataFrame((item.split('=') if len(item.split('='))==2 else ['missing',1]) for item in anom_test.payload)
anom_test_payload.columns = ['keyword','value']
anom_test_payload.head(20)

,keyword,value
0,id,2
1,nombre,Jam�n Ib�rico
2,precio,85
3,cantidad,'; DROP TABLE usuarios; SELECT * FROM datos WH...
4,B1,A�adir al carrito
5,id,2/
6,nombre,Jam�n Ib�rico
7,precio,85
8,cantidad,49
9,B1,A�adir al carrito


In [229]:
normal_test.payload = normal_test.payload.map(str)
normal_test_payload = pd.DataFrame((item.split('=') if len(item.split('='))==2 else ['missing',1]) \
                                   for item in normal_test.payload)
normal_test_payload.columns = ['keyword','value']
#normal_test_payload = normal_test_payload.groupby('keyword')
normal_test_payload.head(20)

,keyword,value
0,missing,1
1,id,1
2,nombre,Jam�n Ib�rico
3,precio,39
4,cantidad,41
5,B1,A�adir al carrito
6,modo,entrar
7,login,caria
8,pwd,egipciaca
9,remember,off


In [181]:
normal_test_payload[normal_test_payload.keyword=='missing'].count()/len(normal_test_payload)

keyword    0.192319
value      0.192319
dtype: float64

In [182]:
anom_test_payload[anom_test_payload.keyword=='missing'].count()/len(anom_test_payload)

keyword    0.063665
value      0.063665
dtype: float64

In [191]:
normal_test_payload.value[normal_test_payload.keyword=='id'].map(int).describe()

count    4000.000000
mean        1.777000
std         0.727601
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         3.000000
Name: value, dtype: float64

In [212]:
anom_test_id = pd.to_numeric(anom_test_payload.value[anom_test_payload.keyword=='id'],errors='coerce')
#anom_test_payload.value[anom_test_payload.keyword=='id'].map(lambda s: int(s.rstrip('/')) if s[-1] == '/' else int(s))

In [217]:
len(anom_test_id[anom_test_id.isnull()])/len(anom_test_id)

0.3415929203539823

### About 1/3 of id's in the anomaly set have strangely formatted numbers whereas all id numbers in the normal test set are well-behaved and are in the range of 1 to 3

In [256]:
anom_test_cantidad = pd.to_numeric(anom_test_payload.value[anom_test_payload.keyword=='cantidad'],errors='coerce')
print('fraction of bad cantidad data',len(anom_test_cantidad[anom_test_cantidad.isnull()])/len(anom_test_cantidad))
anom_test_cantidad.describe()

fraction of bad cantidad data 0.0580817730225449


count    2465.000000
mean       48.329817
std        28.665919
min         0.000000
25%        23.000000
50%        49.000000
75%        73.000000
max        98.000000
Name: value, dtype: float64

In [257]:
normal_test_cantidad = pd.to_numeric(normal_test_payload.value[normal_test_payload.keyword=='cantidad'],errors='coerce')
print('fraction of bad cantidad data',len(normal_test_cantidad[normal_test_cantidad.isnull()])/len(normal_test_cantidad))
normal_test_cantidad.describe()

fraction of bad cantidad data 0.0


count    2000.000000
mean       50.554000
std        28.893226
min         0.000000
25%        26.000000
50%        49.000000
75%        77.000000
max        98.000000
Name: value, dtype: float64

### Presence of bad cantidad data is a weak indicator of an anomaly (5.8% versus 0%)

In [233]:
anom_test_login = anom_test_payload.value[anom_test_payload.keyword=='login']
anom_test_login.head(15)

12      bob@<SCRipt>alert(Paros)</scrIPT>.parosproxy.org
17                                              grimshaw
22                                              grimshaw
40                                                armand
53                                                armand
66                                              bau*gnon
79                                                armand
102                                             thyagara
115                                             thyagara
128                                             thyagara
141                                             thyagara
172    bob%40%3CSCRipt%3Ealert%28Paros%29%3C%2FscrIPT...
177                                             modestin
207                                                 audy
220                                                 audy
Name: value, dtype: object

In [234]:
normal_test_login = normal_test_payload.value[normal_test_payload.keyword=='login']
normal_test_login.head(15)

7         caria
20     minthorn
42     nataraja
69       frayda
82       neddie
104      ye-sho
131     walpole
144       lali8
166     kuswara
193       yaung
206    chopowic
228        caty
255      talbot
268       bizga
290      grogan
Name: value, dtype: object

In [235]:
normal_test_login.describe()

count       6000
unique      2741
top       cullum
freq           6
Name: value, dtype: object

In [236]:
anom_test_login.describe()

count        7894
unique       1684
top       montijo
freq           21
Name: value, dtype: object

In [251]:
len(anom_test_login[anom_test_login.str.contains('\@')])+len(anom_test_login[anom_test_login.str.contains('\.')])

64

In [248]:
len(anom_test_login[anom_test_login.str.contains('%40')])+len(anom_test_login[anom_test_login.str.contains('%3C')])

28

In [243]:
len(normal_test_login[normal_test_login.str.contains('@')])

0

In [244]:
len(normal_test_login[normal_test_login.str.contains('%40')])

0

### Having special characters like "@" or hex equivalent "%40" is a weak indicator of an anomaly

In [252]:
anom_test_precio = pd.to_numeric(anom_test_payload.value[anom_test_payload.keyword=='precio'],errors='coerce')
print('fraction of bad precio data',len(anom_test_precio[anom_test_precio.isnull()])/len(anom_test_precio))
anom_test_precio.describe()

fraction of bad precio data 0.13454913880445796


count    4271.000000
mean     2465.263404
std      3158.182414
min        20.000000
25%        85.000000
50%       100.000000
75%      4975.000000
max      9874.000000
Name: value, dtype: float64

In [253]:
normal_test_precio = pd.to_numeric(normal_test_payload.value[normal_test_payload.keyword=='precio'],errors='coerce')
print('fraction of bad precio data',len(normal_test_precio[normal_test_precio.isnull()])/len(normal_test_precio))
normal_test_precio.describe()

fraction of bad precio data 0.0


count    4000.000000
mean     2519.591000
std      3181.823333
min         2.000000
25%        85.000000
50%       100.000000
75%      4922.250000
max      9895.000000
Name: value, dtype: float64

### Presence of bad presio data is also a weak indicator of an anomaly (13.4% versus 0%)

In [66]:
try:
    cantidad_num_anom_test = cantidad_anom_test.str.lstrip('cantidad=')
    i = cantidad_num_anom_test.map(int)
except ValueError:
    i = None
    print("Could not convert data to an integer.")

Could not convert data to an integer.


In [68]:
print(i)

None


In [272]:
anom_test.url.tail(10)

119569    http://localhost:8080/tienda1/miembros/editar.jsp
119570    http://localhost:8080/tienda1/miembros/editar.jsp
119571    http://localhost:8080/tienda1/miembros/editar.jsp
119572    http://localhost:8080/tienda1/miembros/editar.jsp
119573    http://localhost:8080/tienda1/miembros/editar.jsp
119574    http://localhost:8080/tienda1/miembros/editar.jsp
119575    http://localhost:8080/tienda1/miembros/editar.jsp
119576    http://localhost:8080/tienda1/miembros/editar.jsp
119577    http://localhost:8080/tienda1/miembros/editar.jsp
119578    http://localhost:8080/tienda1/miembros/editar.jsp
Name: url, dtype: object

In [273]:
normal_test.url.tail(10)

103984    http://localhost:8080/tienda1/miembros/editar.jsp
103985    http://localhost:8080/tienda1/miembros/editar.jsp
103986    http://localhost:8080/tienda1/miembros/editar.jsp
103987    http://localhost:8080/tienda1/miembros/editar.jsp
103988    http://localhost:8080/tienda1/miembros/editar.jsp
103989    http://localhost:8080/tienda1/miembros/editar.jsp
103990    http://localhost:8080/tienda1/miembros/editar.jsp
103991    http://localhost:8080/tienda1/miembros/editar.jsp
103992    http://localhost:8080/tienda1/miembros/editar.jsp
103993    http://localhost:8080/tienda1/miembros/editar.jsp
Name: url, dtype: object

### URLs are not an obvious predictor either

In [263]:
normal_test['index'].describe()

count    103994.000000
mean      21268.002981
std       10914.689771
min           0.000000
25%       11741.000000
50%       23483.000000
75%       31047.000000
max       35999.000000
Name: index, dtype: float64

In [265]:
anom_test['index'].describe()

count    119579.000000
mean      13631.813964
std        7276.203104
min           0.000000
25%        7373.500000
50%       14668.000000
75%       19944.000000
max       25064.000000
Name: index, dtype: float64

### Index, on the other hand, is!!!

In [278]:
anom_test.userAgent.head(10)

0    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
1    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
2    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
3    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
4    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
5    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
6    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
7    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
8    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
9    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
Name: userAgent, dtype: object

In [279]:
normal_test.userAgent.head(10)

0    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
1    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
2    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
3    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
4    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
5    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
6    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
7    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
8    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
9    Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...
Name: userAgent, dtype: object

In [281]:
anom_test.protocol.head(10)

0    HTTP/1.1
1    HTTP/1.1
2    HTTP/1.1
3    HTTP/1.1
4    HTTP/1.1
5    HTTP/1.1
6    HTTP/1.1
7    HTTP/1.1
8    HTTP/1.1
9    HTTP/1.1
Name: protocol, dtype: object

In [282]:
normal_test.protocol.head(10)

0    HTTP/1.1
1    HTTP/1.1
2    HTTP/1.1
3    HTTP/1.1
4    HTTP/1.1
5    HTTP/1.1
6    HTTP/1.1
7    HTTP/1.1
8    HTTP/1.1
9    HTTP/1.1
Name: protocol, dtype: object

In [283]:
normal_test.cacheControl.head(10)

0    no-cache
1    no-cache
2    no-cache
3    no-cache
4    no-cache
5    no-cache
6    no-cache
7    no-cache
8    no-cache
9    no-cache
Name: cacheControl, dtype: object

In [284]:
anom_test.cacheControl.head(10)

0    no-cache
1    no-cache
2    no-cache
3    no-cache
4    no-cache
5    no-cache
6    no-cache
7    no-cache
8    no-cache
9    no-cache
Name: cacheControl, dtype: object